#### Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Read TSV files

In [ ]:
df_train = pd.read_csv('data/train.tsv', sep='\t', low_memory=True)
print(df_train.shape)
print(df_train.info())
display(df_train.head())

In [ ]:
df_test = pd.read_csv('data/test.tsv', sep='\t', low_memory=True)
print(df_test.shape)
print(df_test.info())
display(df_test.head())

#### Count the number of words in the product name and product description

In [ ]:
def wordCount(text):
    """
    Parameters:
      text(str): 商品名、商品の説明文
    """
    try:
        if text == 'No description yet':
            return 0  # 商品名や説明が'No description yet'の場合は0を返す
        else:
            text = text.lower()                  # すべて小文字にする
            words = [w for w in text.split(" ")] # スペースで切り分ける
            return len(words)                    # 単語の数を返す
    except: 
        return 0

# 'name'の各フィールドの単語数を'name_len'に登録
df_train['name_len'] = df_train['name'].apply(lambda x: wordCount(x))
df_test['name_len'] = df_test['name'].apply(lambda x: wordCount(x))
# 'item_description'の各フィールドの単語数を'desc_len'に登録
df_train['desc_len'] = df_train['item_description'].apply(lambda x: wordCount(x))
df_test['desc_len'] = df_test['item_description'].apply(lambda x: wordCount(x))

In [ ]:
display(df_train.head())

In [ ]:
display(df_test.head())

#### Take a look at the 'price'

In [ ]:
df_train['price'].describe()

In [ ]:
#priceをヒストグラムで表示
sns.set()
plt.figure(figsize=(10, 5))
plt.hist(df_train['price'], bins=50, range=[0, 150], alpha=0.7, color='b', label='price')
plt.title('Price Distribution')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.legend()

#### Scaling the selling price

In [ ]:
# 訓練データの'price'を対数変換する
df_train["target"] = np.log1p(df_train.price)
# ヒストグラムを表示
plt.figure(figsize=(10, 5))
plt.hist(df_train["target"], bins=50, range=[0, 7], alpha=0.7, color='b', label='price')
plt.title('Price Distribution(Log Scale)')
plt.xlabel('log(Price)')
plt.ylabel('Frequency')
plt.legend(['log(price)'])

#### Splitting the category name and registering it in a new column

In [ ]:
def split_cat(text):
    """
    Parameters:
      text(str): カテゴリ名

    ・カテゴリを/で切り分ける
    ・データが存在しない場合は"No Label"を返す
    """
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")

# 3つに切り分けたカテゴリ名を'subcat_0'、'subcat_1'、'subcat_2'に登録
# 訓練データ
df_train['subcat_0'], df_train['subcat_1'], df_train['subcat_2'] = \
    zip(*df_train['category_name'].apply(lambda x: split_cat(x)))
# テストデータ
df_test['subcat_0'], df_test['subcat_1'], df_test['subcat_2'] = \
    zip(*df_test['category_name'].apply(lambda x: split_cat(x)))

In [ ]:
display(df_train.head())

In [ ]:
display(df_test.head())

#### Replace the missing values of the brand name with meaningful data

In [ ]:
# df_trainとdf_testを縦方向に結合
full_set = pd.concat([df_train, df_test])
# full_setの'brand_name'から重複なしのブランドリスト(集合)を生成
all_brands = set(full_set['brand_name'].values)

# 'brand_name'の欠損値NaNを'missing'に置き換える
df_train['brand_name'].fillna(value='missing', inplace=True)
df_test['brand_name'].fillna(value='missing', inplace=True)

# 訓練データの'brand_name'が'missing'に一致するレコード数を取得
train_premissing = len(df_train.loc[df_train['brand_name'] == 'missing'])
# テストデータの'brand_name'が'missing'に一致するレコード数を取得
test_premissing = len(df_test.loc[df_test['brand_name'] == 'missing'])

def brandfinder(line):
    """
    Parameters: line(str): ブランド名

    ・ブランド名の'missing'を商品名に置き換える:
         missing'の商品名の単語がブランドリストに存在する場合
    ・ブランド名を商品名に置き換える:
        商品名がブランドリストの名前と完全に一致する場合
    ・ブランド名をそのままにする:
        商品名がブランドリストの名前と一致しない
        商品名が'missing'だが商品名の単語がブランドリストにない
    """
    brand = line[0] # 第1要素はブランド名
    name = line[1]  # 第2要素は商品名
    namesplit = name.split(' ') # 商品名をスペースで切り分ける
    
    if brand == 'missing':  # ブランド名が'missing'と一致
        for x in namesplit: # 商品名から切り分けた単語を取り出す
            if x in all_brands:                
                return name # 単語がブランドリストに一致したら商品名を返す
    if name in all_brands:  # 商品名がブランドリストに存在すれば商品名を返す
        return name
    
    return brand            # どれにも一致しなければブランド名を返す

# ブランド名の付替えを実施
df_train['brand_name'] = df_train[['brand_name','name']].apply(brandfinder, axis = 1)
df_test['brand_name'] = df_test[['brand_name','name']].apply(brandfinder, axis = 1)

# 書き換えられた'missing'の数を取得
train_found = train_premissing-len(df_train.loc[df_train['brand_name'] == 'missing'])
test_found = test_premissing-len(df_test.loc[df_test['brand_name'] == 'missing'])
print(train_premissing) # 書き換える前の'missing'の数
print(train_found)      # 書き換えられた'missing'の数
print(test_premissing)  # 書き換える前の'missing'の数
print(test_found)       # 書き換えられた'missing'の数

In [ ]:
display(df_train.head())